In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import igraph as ig
import scanpy as sc
import scanpy.external as sce
import triku as tk

#Plotting
import seaborn as sns
import matplotlib.pyplot as plt

#utils
from datetime import datetime
from scipy.sparse import csr_matrix, isspmatrix

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80)
time_start = datetime.now()
print(time_start)

2023-11-03 03:57:23.933730


In [3]:
import sys
sys.path.append('../..')

from src.configs import config
from src.utils import utils as us
from src.utils import visualise as vs
%load_ext autoreload
%autoreload 2
warnings.filterwarnings("once")

# Data Load

In [4]:
adata = sc.read(config.PATHS.LOGS/'QC.h5ad')
us.log(adata.shape)
us.log('isspmatrix: {}'.format(isspmatrix(adata.X)))


us.log('Loaded Filtered AnnData object: number of cells {}'.format(
    adata.n_obs))
us.log('Loaded Filtered AnnData object: number of genes {}'.format(
    adata.n_vars) )
us.log('Available metadata for each cell:  {}'.format(adata.obs.columns))

(24549, 15223)
isspmatrix: True
Loaded Filtered AnnData object: number of cells 24549
Loaded Filtered AnnData object: number of genes 15223
Available metadata for each cell:  Index(['timepoint', 'line', '10X_date', 'clusters', 'tSNE_1', 'tSNE_2',
       'cluster', 'timepoint_mapped', 'batch', 'dataset', 'isHuman', 'isESC',
       'n_genes_by_counts', 'total_counts', 'total_counts_mito',
       'pct_counts_mito', 'total_counts_ribo', 'pct_counts_ribo',
       'QC_doublets', 'n_genes', 'n_counts'],
      dtype='object')


# Normalize and Log Transform data: Scanpy Normalization

Basic Scanpy CPM Normalization

In [5]:
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6, exclude_highly_expressed=True)
sc.pp.log1p(adata)

normalizing counts per cell The following highly-expressed genes are not considered during normalization factor computation:
['ID3', 'BCYRN1', 'TMSB10', 'SST', 'AFP', 'ACTB', 'TMSB4X', 'TIMP1', 'IGF2', 'MDK', 'FTH1', 'MALAT1', 'GAL', 'APOA1', 'TAC3', 'DLK1', 'CRABP1', 'APOE', 'FTL', 'MGP', 'LUM', '7SK.2']
    finished (0:00:00)


Store normalized counts

In [6]:
adata.layers['lognormcounts'] = adata.X.copy()


Alternative workflow: normalization by Scran
References: Scran Paper | Scran R Vignette | Theis Scran Tutorial in scanpy |

Normalizing cell-specific biases

Cell-specific biases are normalized using the computeSumFactors() method, which implements the deconvolution strategy for scaling normalization (A. T. Lun, Bach, and Marioni 2016). This computes size factors that are used to scale the counts in each cell. The assumption is that most genes are not differentially expressed (DE) between cells , such that any differences in expression across the majority of genes represents some technical bias that should be removed.


# Feature selection: Triku
Sources: Triku Paper | Docs |

The premise of triku is that, for genes with similar expression levels, the expression pattern can be categorized in three states:

    i: the gene is expressed throughout the cells with similar expression levels (a): NO useful information about specific cell types associated to that gene
    ii: the expression of the gene can be localized in a subset of cells, which can in turn be:
        Transcriptomically different cells (b1) (i.e. cells that are not neighbours in the dimensionally reduced map)
        Transcriptomically similar cells (b2) (neighbours): the gene is more probably biologically relevant for that population

Triku aims to select genes of case (b2) while avoiding the selection of genes of case (a) and (b1).

It does so by looking at the expression in the nearest neighbours
PROs of this method:

    selects more biologically relevant genes
    avoids selection of mitocondrial and ribosomal genes
    tends to select a lower number of HVGs aiding downstream computations

The Algorithm

    Create a neighbour graph by selecting the k cells with the most similar transcriptome to each cell in the dataset kNN:
    For each gene:
        Obtain the distribution of the the kNN counts summing the counts of each cell and its neighbors for each cell with positive expression in the dataset.
        Simulate a null distribution, as above but considering k random cells instead of kNN ones.
        Compare the kNN distribution with its corresponding null distribution ì
        Compute the Wasserstein distance between both distributions
    Normalize Wasserstein distances
    Select the features with highest Wasserstein distance.


## Compute neighbors

In [7]:
sc.pp.pca(adata, use_highly_variable=False)
# specify in case you want to try different HVG selection methods 
sc.pp.neighbors(
    adata, metric='cosine', n_neighbors=int(0.5 * len(adata) ** 0.5)) 

computing PCA
    with n_comps=50
    finished (0:00:39)
computing neighbors
    using 'X_pca' with n_pcs = 50


/home/vess/Distr/conda/envs/HT/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/vess/Distr/conda/envs/HT/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vess/Distr/conda/envs/HT/lib/python3.9/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to Tru

    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:47)


## Identify HVG

In [8]:
if config.PROTO.SUBSET.BATCH_KEY is not None:
    groups = adata.obs.groupby(config.PROTO.SUBSET.BATCH_KEY).groups
    print({ 
        k: adata.obs.loc[idxs].line.unique() 
        for k, idxs in groups.items()})

{170921: ['409b2']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 171122: ['409b2', 'sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 171212: ['409b2', 'sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 180221: ['409b2', 'sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 180321: ['409b2', 'sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 180410: ['409b2', 'h9', 'sc102a1']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 190327: ['sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 190507: ['sandraA']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 200818: ['sc102a1', 'h9', 'joc']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 200908: ['joc']
Categories (5, object): ['409b2', 'h9', 'joc', 'sandraA', 'sc102a1'], 200920: ['joc', 'h9', 'sc102a1']
Categori

In [9]:
if config.PROTO.SUBSET.HVG_ALGOS == 'triku':
    tk.tl.triku(
        adata, use_raw=False,
        n_features=config.PROTO.SUBSET.NB_HIGHLY_VARIABLE)
    us.log(
        'Number of Higly Variable Genes',
        len(adata.var_names[adata.var['highly_variable'] == True]))

    Top20Triku = adata.var.sort_values(
        by=['triku_distance'], ascending=False).head(20).index
    print(Top20Triku)

Number of Higly Variable Genes 5000
Index(['PGAM4', 'TDGF1', 'RP11-466P24.7', 'C9orf135', 'SMPDL3B', 'LINC00678',
       'RP4-792G4.2', 'ZSCAN10', 'RP11-1144P22.1', 'CLDN7', 'NANOG', 'BGN',
       'SCGB3A2', 'FOXH1', 'GPR160', 'RP11-277L2.5', 'CDH1', 'POU5F1', 'PRODH',
       'UTF1'],
      dtype='object', name='Unnamed: 0')


In [10]:
if config.PROTO.SUBSET.HVG_ALGOS == 'hvgsc':
    adata.obs[config.PROTO.SUBSET.BATCH_KEY] = adata.obs[
        config.PROTO.SUBSET.BATCH_KEY].astype(str)
    sc.pp.highly_variable_genes(
        adata, n_top_genes=config.PROTO.SUBSET.NB_HIGHLY_VARIABLE,
        batch_key=config.PROTO.SUBSET.BATCH_KEY,
    )
    sc.pl.highly_variable_genes(adata)

    us.log(
        'Number of Higly Variable Genes', 
        len(adata.var_names[adata.var['highly_variable'] == True]))

Batch effect could influence HVG selection.

You can consider to correct (e.g. with Harmony) the neighbors used to compute the HVGs. For this dataset we did not notice a big difference between the two procedures.  

# Save

In [11]:
try:
    del adata.uns['triku_params']
except: pass

adata.write(config.PATHS.LOGS/'Norm.h5ad')
us.log(datetime.now() - time_start)

0:05:02.183112
